# Initial Setup

Mentions: 
- Thank you for emasters for their excellent [tutorial](https://www.youtube.com/watch?v=DLngCtsG3bk)

In [ ]:
# Check for Nvidia GPU. Required for cudaNN
!nvidia-smi

Thu Nov 25 07:04:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#imports
import cv2, os
import numpy as np
import matplotlib.pyplot as plt
import math
import decimal
import copy
import glob

# Data Prep

### Project Repo with modified Darknet

In [ ]:
!git clone https://github.com/HughN7/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15368, done.
remote: Total 15368 (delta 0), reused 0 (delta 0), pack-reused 15368
Receiving objects: 100% (15368/15368), 14.00 MiB | 13.58 MiB/s, done.
Resolving deltas: 100% (10334/10334), done.


In [ ]:
#Configure Darknet to enable cv, gpu and cudann
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


#### Before compiling. Replace the detector.c file under darknet/src with the detector.c file on the github repo. This will save the weights every 25 epochs/cycles

In [ ]:
#Compile Darknet
!make

In [ ]:
#Configure yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg #Copy two files

!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg

#2configure two classes
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg

#1 class at 18 filters while 2 classes at 21
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

In [ ]:
!echo -e 'Mask\nNoMask' > data/obj.names #.names save to obj.names

In [ ]:
#.data = 5 lines:
# number of classes | classes= 2\n
# path to train.txt | train = data/train.txt\n
# path to test.txt | valid  = data/test.txt\n
# path to obj.names | names = data/obj.names\n
# path to save trained weights = backup = /content/drive/MyDrive/Colab Notebooks/Senior Project/model'
!echo -e 'classes= 2\ntrain = data/train.txt\nvalid = data/train.txt\nnames = data/obj.names\nbackup = /content/drive/MyDrive/SeniorProject/maskmodel' > data/obj.data

In [ ]:
#unzip training data
!mkdir data/obj
!unzip /content/drive/MyDrive/SeniorProject/MaskTrainingFiles/masks.zip -d data/obj

# Training

### Create train.txt

In [ ]:
#Tell which images + path that yolo needs to be trained on
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

### Download Pre-trained weights

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

### Execute
- darknet will create a chart.png file under darknet folder. Download the file probably every 100 cycles
- if colab stops for some reason. Use the second command and comment out the first one to continue training. 

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 2>&1 > /content/drive/MyDrive/SeniorProject/maskmodel/log.txt -dont_show -map

In [ ]:
#use this in the event colab kicks you out (free tier)
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /content/drive/MyDrive/SeniorProject/maskmodel/yolov3_training_last.weights > log.txt -dont_show -map